ЦЕЛЬ
Подготовить основу рекомендательной системы, благодаря которой можно будет предлагать клиентам интересные им курсы и тем самым повышать средний чек.

ЗАДАЧИ

1. Подготовить данные для анализа
2. Разбить данные на пары
3. Отобрать к каждому купленному курсу пару рекомендумых курсов

1. Формируем SQL-запрос в Metabase для дальнейшего анализа.
Формируем запрос с данными по продажам курсов в разрезе пользователей (купивших более одного курса)

 with new as( --создаем временную таблицу с данными в разрезе пользоватей и курсов, которые купили пользователи
 select 
 c.user_id,-- выбираем пользователя
 resource_id new_resource_id --выбираем курс
 from final.carts c join final.cart_items ci on c.id = ci.cart_id --соединяем данные из разных таблиц данных по условию
 where c.state = 'successful' and ci.resource_type = 'Course' --отсортировываем только купленные курсы
 group by 1,2), --группируем по первому и ворому столбцу
 
 new1 as( --создаем временную таблицу с данными в разрезе пользоватей купивших больше одного раза
 select 
 c.user_id,-- выбираем пользователя
 count(ci.id) --подсчитываем количество покупок
 from final.carts c join final.cart_items ci on c.id = ci.cart_id --соединяем данные из разных таблиц данных по условию
 where c.state = 'successful' and ci.resource_type = 'Course' --отсортировываем только купленные курсы
 group by 1 --группируем по первому столбцу
 having count(distinct ci.resource_id) >1) --выбираем пользователей совершивших покупку курсов более 1 раза
 
 select --формируем таблицу с данными по продажам курсов в разрезе пользователей (купивших более одного курса)
 new1.user_id, --выбираем пользователей
 new_resource_id --выбираем курсы
 from new1 join new on new1.user_id=new.user_id --соединяем данные из разных таблиц данных по условию
 group by 1,2 --группируем по певому и второму столбцу
 order by 1 --сортируем по первому столбцу


Выгружаем полученный запрос в файл формата .csv
Загружаем файл .csv в Visual Studio Code для дальнейшего анализа на языке  Python  в формате файла с расширением .ipynb

In [183]:
import pandas as pd
##import matplotlib.pyplot as plt
df = pd.read_csv('query_result _new.csv')
df.head()

,user_id,new_resource_id
0,51,516
1,51,1099
2,6117,356
3,6117,1125
4,6117,357


Удаляем дубликаты пар в разрезе пользователь-курс

In [184]:
df=df.drop_duplicates()

Группируем курсы по пользователям в списки

In [185]:
group_df=df.groupby('user_id')['new_resource_id'].apply(lambda x:list(pd.unique(x))).reset_index()
group_df.head()

,user_id,new_resource_id
0,51,"[516, 1099]"
1,6117,"[356, 1125, 357]"
2,10275,"[1147, 553]"
3,10457,"[1138, 361]"
4,17166,"[356, 357]"


Разбиваем списки на уникальные пары

In [186]:
import itertools
list_courses = list()
for resouce in group_df['new_resource_id']:
    for pair in itertools.combinations(resouce,2):
        list_courses.append(pair)
set=pd.DataFrame(list_courses,
columns=['№course', 'resouce']) ##формируем датафрайм и присваиваем столбцам названия
set.head()

,№course,resouce
0,516,1099
1,356,1125
2,356,357
3,1125,357
4,1147,553


Группируем пары по уникальности и подсчитываеем количество каждой уникальной пары в датафрайме и отсортируем по столбцу с количеством в порядке убывания. 
Видим, что больше всего купили пар курсов 551-566

In [187]:
es=set.groupby(['№course', 'resouce']).size().reset_index().rename(columns={0:'count'})
es.sort_values(['count'], ascending=[False], kind="heapsort", inplace=True)
es.head()

,№course,resouce,count
1811,551,566,411
2272,566,551,386
1292,515,551,214
1800,551,515,203
1789,551,489,162


Сортируем датафрайм по первому столбцу в порядке возрастания и по третьему столбцу в пордкеубывания
Создаем столбец с уникальным номером курса разрезе курса (столбец один) и ранжируем курсы по количеству покупок пар

In [188]:
es.sort_values(['№course','count'], ascending=[True, False], kind="heapsort", inplace=True)
es['domainId'] = es.groupby('№course')['count'].rank(ascending=False, method='first')
print(es)

      №course  resouce  count  domainId
22        356      571     54       1.0
0         356      357     51       2.0
15        356      551     23       3.0
52        356     1125     22       4.0
34        356      765     15       5.0
...       ...      ...    ...       ...
6363     1199      752      1       3.0
6362     1199      523      1       4.0
6365     1200     1104      1       1.0
6364     1200      794      1       2.0
6366     1201     1125      1       1.0

[6367 rows x 4 columns]


Поскольку есть пары курсов, которые были куплены один раз, нужно определить сколько количество покупок считать значимым для формирования рекомендации.
Найдем среднее значение по столбцу количество

In [189]:
es['count'].mean()

6.285063609235118

Найдем стандартное отклонение

In [190]:
res_std = es['count'].std()
print(res_std)

14.57551867035126


Найдем доверительный интервал для среднего значения купленных курсов и для дальнейших расчетов возьмем целое наименьшее значение этого интервала. Оно равно шести.

In [191]:
from scipy.stats import norm
import math
alpha = 0.01
sigma = 14.58
n = 6367
mean = 6.29
def confidence_interval_norm(alpha, sigma, n, mean):
    value = -norm.ppf(alpha / 2) * sigma / math.sqrt(n)
    return mean - value, mean + value
print(confidence_interval_norm(alpha, sigma, n, mean))
    


(5.819340121208304, 6.760659878791696)


Заменим курсы в парах купленных менее шести раз на популярные 551 и 566. 
Воспользумся для этого столбцом ранжирования. Поскольку нас интересует только самые популярные из купленных пар, то мы применим правило только к 1 и 2 показателю ранжирования.

In [192]:
def itog(row):
    if (row['count']< 6) & (row['domainId']==1):
        row.resouce2 = 551
        return row.resouce
    elif (row['count']< 6) & (row['domainId']==2):
        row.resouce2 = 566
        return row.resouce
    else:
        return row.resouce
new_itog = es.apply(lambda row: itog(row), axis=1)  
es['itog'] = new_itog
es.head(7)

,№course,resouce,count,domainId,itog
22,356,571,54,1.0,571.0
0,356,357,51,2.0,357.0
15,356,551,23,3.0,551.0
52,356,1125,22,4.0,1125.0
34,356,765,15,5.0,765.0
9,356,514,15,6.0,514.0
46,356,912,13,7.0,912.0


Преобразуем наши данные с помощью сводной таблицы. 
Создадим столбы в разрезе курсов по признаку ранжирования.

In [193]:
pivot = es.pivot_table(values='itog', index = '№course', columns = 'domainId',aggfunc='min')
pivot = pivot.copy()
pivot.head()

domainId,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0
№course,,,,,,,,,,,,,,,,,,,,,
356,571.0,357.0,551.0,1125.0,765.0,514.0,912.0,361.0,515.0,566.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,571.0,356.0,1125.0,551.0,765.0,566.0,514.0,553.0,912.0,745.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,570.0,752.0,507.0,359.0,551.0,508.0,1115.0,552.0,523.0,514.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,358.0,570.0,752.0,523.0,507.0,551.0,566.0,519.0,515.0,489.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
360,553.0,745.0,568.0,1099.0,516.0,523.0,1187.0,357.0,679.0,489.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Отберем из полученной сводной таблицы только столбцы с ранжирование 1 и 2.
Преобразем сводную талицу в датафрайм и переименуем столбцы.
Заменим пустые ячейки с данными второй рекомендации на ноль, а затем присвоим значение популярного курса (551). Нули получились в связи с тем, что к курсу в первом столбце ни разу не покупалось других курсов, кроме указанных в столбце два. Возможно это специфическое направление обучения, тк другими курсами покупатели не заинтересовались. В этом случае есть вероятность того, что пользователи данных курсов не заинтересуются рекомендациями, поэтому возможно есть смысл рассмотреть вариант с расширением количества курсов по специфическому направлению.

In [194]:
import numpy as np
pivot_new=pivot.iloc[:, 0:2]
pivot_new = pd.DataFrame(pivot_new.to_records()).rename(columns={'1.0':'recommendation1','2.0':'recommendation2'})
pivot_new = pivot_new.fillna(0)
pivot_new['recommendation2'] = np.where((pivot_new['recommendation2'] == 0), 551, pivot_new['recommendation2'])
print(pivot_new)

     №course  recommendation1  recommendation2
0        356            571.0            357.0
1        357            571.0            356.0
2        358            570.0            752.0
3        359            358.0            570.0
4        360            553.0            745.0
..       ...              ...              ...
121     1188            513.0           1141.0
122     1198            912.0            551.0
123     1199            515.0            519.0
124     1200           1104.0            794.0
125     1201           1125.0            551.0

[126 rows x 3 columns]


Таблица с рекомендациями готова.

PS: Структура SQL запроса в данном файле сильно искажается. Прошу не судить строго.